In [1]:
import numpy as np
import sys
import os
import pickle
import torch
sys.path.append(os.getcwd())
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from rus import *
import pandas as pd

#### process results csv

In [20]:
df = pd.read_csv('synthetic/model_selection/results.csv')
df.keys()

Index(['dataset', ' model', ' R', ' U1', ' U2', ' S', ' acc'], dtype='object')

In [21]:
SETTINGS = np.unique(df['dataset'])
METHODS = ['agree', 'align', 'early_fusion', 'elem', 'mfm', 'mi', 'mult', 'outer', 'lower']
MEASURES = ['redundancy', 'unique1', 'unique2', 'synergy']

In [22]:
datasets = dict()
results = dict()
for setting in SETTINGS:
    results[setting] = dict()
for i in range(len(df)):
    entry = df.loc[i]
    if entry[' model'] == ' dataset':
        datasets[entry['dataset'].strip()] = {'redundancy':entry[' R'], 'unique1':entry[' U1'], 'unique2':entry[' U2'], 'synergy':entry[' S']}
    else:
        results[entry['dataset'].strip()][entry[' model'].strip().lower()] = {'redundancy':entry[' R'], 'unique1':entry[' U1'], 'unique2':entry[' U2'], 'synergy':entry[' S'], 'acc':entry[' acc']}
    
with open('synthetic/model_selection/results_affect.pickle', 'wb') as f:
    pickle.dump(results, f)
with open('synthetic/model_selection/datasets_affect.pickle', 'wb') as f:
    pickle.dump(datasets, f)

In [23]:
with open('synthetic/model_selection/results_affect.pickle', 'rb') as f:
    results = pickle.load(f)
print(results.keys())
with open('synthetic/model_selection/datasets_affect.pickle', 'rb') as f:
    datasets = pickle.load(f)
print(datasets.keys())
normalized_results = dict()
normalized_datasets = dict()
accs = []
for setting in SETTINGS:
    normalized_results[setting] = dict()
    for method in results[setting]:
        normalized_results[setting][method] = dict()
        normalized_datasets[setting] = dict()
        results_total = sum([results[setting][method][measure] for measure in results[setting][method]])
        datasets_total = sum([datasets[setting][measure] for measure in datasets[setting]])
        for measure in datasets[setting]:
            normalized_results[setting][method][measure] = results[setting][method][measure] / results_total
            normalized_datasets[setting][measure] = datasets[setting][measure] / datasets_total
        accs.append(results[setting][method]['acc'])

dict_keys(['enrico', 'humor01', 'humor02', 'humor12', 'mimic', 'mosei01', 'mosei02', 'mosei12', 'sarcasm01', 'sarcasm02', 'sarcasm12'])
dict_keys(['mimic', 'enrico', 'humor12', 'humor01', 'humor02', 'mosei12', 'mosei01', 'mosei02', 'sarcasm01', 'sarcasm02', 'sarcasm12'])


In [5]:
for dataset in datasets:
    print(dataset, datasets[dataset])

mimic {'redundancy': 0.0513069475394659, 'unique1': 0.1708080626662141, 'unique2': 2.719328957684913e-08, 'synergy': 0.0056557956533846}
enrico {'redundancy': 0.7305898871257488, 'unique1': 0.3756550453895989, 'unique2': 0.5295818376620958, 'synergy': 0.3351161376596552}
humor12 {'redundancy': 0.0074837124438409, 'unique1': 0.0021428311352223, 'unique2': 0.0018722723944138, 'synergy': 0.0380261035251354}
humor01 {'redundancy': 0.0096185016837852, 'unique1': 0.0215670453302197, 'unique2': 8.041879712466572e-06, 'synergy': 0.1739730317062034}
humor02 {'redundancy': 0.0093378549487458, 'unique1': 0.0218476920659914, 'unique2': 1.81299020742313e-05, 'synergy': 0.1684954904734209}
mosei12 {'redundancy': 0.0122012771897019, 'unique1': 5.1115889436551705e-15, 'unique2': 0.0079621148751152, 'synergy': 0.0237073327557628}
mosei01 {'redundancy': 0.0119234634791755, 'unique1': 0.0047424614051067, 'unique2': 0.0002778137128733, 'synergy': 0.0227675033158421}
mosei02 {'redundancy': 0.01487155384336

In [27]:
model_selection_best = dict()
for setting in datasets:
    measures = []
    for method in results[setting]:
        measures.append((results[setting][method]['acc'], method))
    measures = sorted(measures)
    model_selection_best[setting] = results[setting][measures[-1][1]]['acc']
    print(setting, measures[-5:])

mimic [(0.9144696505211528, 'outer'), (0.91477621091355, 'align'), (0.9163090128755365, 'additive'), (0.9163090128755365, 'mi'), (0.9172286940527284, 'agree')]
enrico [(0.496575342465753, 'outer'), (0.5, 'early_fusion'), (0.5068493150684932, 'agree'), (0.517123287671232, 'align'), (0.5205479452054794, 'lower')]
humor12 [(0.6030245746691871, 'lower'), (0.615311909262759, 'mfm'), (0.619092627599243, 'outer'), (0.620982986767485, 'mi'), (0.636105860113421, 'elem')]
humor01 [(0.592627599243856, 'early_fusion'), (0.594517958412098, 'elem'), (0.595463137996219, 'mfm'), (0.598298676748582, 'outer'), (0.609640831758034, 'mi')]
humor02 [(0.622873345935727, 'outer'), (0.623818525519848, 'early_fusion'), (0.624763705103969, 'lower'), (0.64461247637051, 'mi'), (0.648393194706994, 'mfm')]
mosei12 [(0.7984602694528458, 'outer'), (0.799285125103107, 'additive'), (0.800659884520209, 'elem'), (0.8012097882870498, 'agree'), (0.8100082485565027, 'mi')]
mosei01 [(0.6464118779213638, 'additive'), (0.646961

In [6]:
with open('synthetic/experiments2/datasets.pickle', 'rb') as f:
    old_datasets = pickle.load(f)
with open('synthetic/experiments2/results.pickle', 'rb') as f:
    old_results = pickle.load(f)
normalized_old_datasets = dict()
for setting in old_datasets:
    normalized_old_datasets[setting] = dict()
    datasets_total = sum([old_datasets[setting][measure] for measure in MEASURES])
    for measure in MEASURES:
        normalized_old_datasets[setting][measure] = old_datasets[setting][measure] / datasets_total
methods = ['additive', 'agree', 'align', 'early_fusion', 'elem', 'recon', 'mi', 'mult', 'outer', 'lower']
model_selection = dict()
for setting in old_datasets:
    measures = sorted([(old_results[method][setting]['acc'], method) for method in methods])
    model_selection[setting] = measures[-3:]

In [31]:
for setting in datasets:
    tmp = []
    for old_setting in old_datasets:
        tmp.append((np.sum(np.absolute([normalized_datasets[setting][measure]-normalized_old_datasets[old_setting][measure] for measure in MEASURES])), old_setting))
    tmp = sorted(tmp)
    for i in range(1):
        print(setting, "selected models", model_selection[tmp[i][1]])
        selected_models = model_selection[tmp[i][1]]
        model_scores = []
        for _, method in selected_models:
            if method == 'recon':
                method = 'mfm'
            if method in results[setting]:
                model_scores.append((results[setting][method]['acc'], method))
        model_scores = sorted(model_scores)
        print(model_scores[-1][0] / model_selection_best[setting] * 100)
print(model_selection_best)

mimic selected models [(0.7316666666666667, 'outer'), (0.7338888888888889, 'early_fusion'), (0.7361111111111112, 'recon')]
99.69919786096257
enrico selected models [(0.7138888888888889, 'additive'), (0.7183333333333334, 'align'), (0.72, 'agree')]
99.34210526315772
humor12 selected models [(0.5461111111111111, 'recon'), (0.5538888888888889, 'align'), (0.555, 'mult')]
96.73105497771168
humor01 selected models [(0.5461111111111111, 'recon'), (0.5538888888888889, 'align'), (0.555, 'mult')]
97.67441860465112
humor02 selected models [(0.5461111111111111, 'recon'), (0.5538888888888889, 'align'), (0.555, 'mult')]
100.0
mosei12 selected models [(0.6477777777777778, 'agree'), (0.6488888888888888, 'align'), (0.6488888888888888, 'recon')]
98.91378139850644
mosei01 selected models [(0.6844444444444444, 'early_fusion'), (0.685, 'recon'), (0.6877777777777778, 'agree')]
96.41652613827993
mosei02 selected models [(0.6477777777777778, 'agree'), (0.6488888888888888, 'align'), (0.6488888888888888, 'recon'